In [1]:
import numpy as np
import io
import pandas as pd
import os
import fnmatch
import shutil
import re
from ruamel.yaml import YAML


In [22]:
import os

directory_path = '/braintree/home/aliya277/inventory_new'

for  dirpath, dirnames, filenames in os.walk(directory_path):   
    try: 
        for names in dirnames:
            dates = os.listdir(dirpath+'/'+names+'/'+os.listdir(dirpath+'/'+names)[0])

            for date in dates:
                counter = 0
                files = os.listdir(dirpath+'/'+names+'/'+os.listdir(dirpath+'/'+names)[0]+'/'+date)
                for file in files:
                    if file.endswith('.proc'): counter +=1
                if counter >1 and 'h5' in files:
                    path = dirpath+'/'+names+'/'+os.listdir(dirpath+'/'+names)[0]+'/'+date+'/
                    print(counter, date, files)
    except: pass

2 norm_FOSS.sub_pico.20230510 ['norm_FOSS.sub_pico.20230510_125550.proc', 'h5', 'norm_FOSS.sub_pico.20230510_111142.proc']
2 exp_muri1320-2023-v1.sub_pico.20230127 ['exp_muri1320-2023-v1.sub_pico.20230127_161915.proc', 'h5', 'exp_muri1320-2023-v1.sub_pico.20230127_172414.proc']
2 exp_muri1320-2023-v1.sub_pico.20230130 ['exp_muri1320-2023-v1.sub_pico.20230130_142333.proc', 'h5', 'exp_muri1320-2023-v1.sub_pico.20230130_144347.proc']
2 exp_muri1320-2023-v1.sub_pico.20230131 ['exp_muri1320-2023-v1.sub_pico.20230131_152020.proc', 'h5', 'exp_muri1320-2023-v1.sub_pico.20230131_183129.proc']
2 exp_muri1320-2023-v1.sub_pico.20230201 ['exp_muri1320-2023-v1.sub_pico.20230201_135241.proc', 'h5', 'exp_muri1320-2023-v1.sub_pico.20230201_152725.proc']
2 exp_muri1320-2023-v0.sub_pico.20230125 ['exp_muri1320-2023-v0.sub_pico.20230125_150432.proc', 'h5', 'exp_muri1320-2023-v0.sub_pico.20230125_161525.proc']
2 exp_muri1320-2023-v0.sub_pico.20230124 ['exp_muri1320-2023-v0.sub_pico.20230124_140420.proc', '

In [2]:
df = pd.read_excel( '/braintree/home/aliya277/dandi_brainscore/pico_inventory_new.xlsx'  )

In [11]:
for ele in df['Path: h5']:
    if ele != np.nan: print(ele)

nan
/braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/h5/220616.pico.rsvp.normalizers.experiment_psth_raw.h5
nan
nan
nan
nan
nan
nan
/braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/h5/220615_pico.rsvp.normalizers.experiment_psth_raw.h5
nan
nan
nan
nan
/braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/h5/220907.pico.rsvp.normalizers.experiment_psth_raw.h5
/braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/h5/220929.pico.rsvp.normalizers.experiment_psth_raw.h5
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
/braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/h5/230328.pico.rsvp.normalizers.experiment_psth_raw.h5
nan
/braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/h5/230428.pico.rsvp.normalizers.experiment_psth_raw.h5
/braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/h5/2305

In [16]:
############### Check Directories #############################################
###############################################################################

df = pd.read_excel( '/braintree/home/aliya277/dandi_brainscore/pico_inventory_new.xlsx'  )
SubjectName = 'pico'
storage_dir = '/braintree/home/aliya277/inventory'

def same_folders(dcmp):
    if dcmp.diff_files or dcmp.left_only or dcmp.right_only:
        return False
    for sub_dcmp in dcmp.subdirs.values():
        if not same_folders(sub_dcmp):
            return False
    return True



for imageset, excelname, date_, time_, hasspike, haspsth, hash5, spikepath, psthpath, h5path in zip(df['ImageSet'], df['(excel) Stimuli'], df['date'], df['time'], df['Has SpikeTime'], \
    df['Has psth'], df['Has h5'], df['Path: SpikeTimes'], df['Path: psth'], df['Path: h5']):
    date = f'20{date_}'
    if len(str(time_)) != 6: time = f'0{time_}'
    else: time = time_
    if hasspike == 1:
        # check if directory in created inventory exists
        if imageset == 'normalizers':
            directory = f'norm_FOSS.sub_{SubjectName}.{date}_{time}.proc'
        elif imageset == 'normalizers-HVM':
            directory = f'norm_HVM.sub_{SubjectName}.{date}_{time}.proc'
        else: 
            directory = f'exp_{imageset}.sub_{SubjectName}.{date}_{time}.proc'
            directory_excel = f'exp_{excelname}.sub_{SubjectName}.{date}_{time}.proc'
        
        imagesetdir = os.path.join(storage_dir, ".".join(directory.split(".")[0:1]))
        subjectdir  = os.path.join(storage_dir, imagesetdir, ".".join(directory.split(".")[0:2]))
        subjectdir_date  = os.path.join(subjectdir, ".".join(directory.split(".")[0:2])+'.'+date)
        print(imagesetdir)
        print(subjectdir)
        print(subjectdir_date)
            
        try: os.mkdir(imagesetdir)
        except: pass
        try: os.mkdir(subjectdir)
        except: pass
        try: os.mkdir(subjectdir_date)
        except: pass
        
        if os.path.isdir(subjectdir+'/'+directory): 
            try: os.mkdir(subjectdir_date+'/'+directory)
            except: pass
    
            # Check if has SpikeTimes
            # if len(os.listdir(subjectdir+'/'+directory+'/'+'SpikeTimes'))!=0: 
            #     print(f'Moving SpikeTimes files for {directory}')
                # shutil.copytree((subjectdir+'/'+directory+'/'+'SpikeTimes'), os.path.join(subjectdir_date, directory, 'SpikeTimes'))
                # shutil.rmtree(subjectdir+'/'+directory+'/'+'SpikeTimes')

            #Check if has h5
            if hash5 == 1:
                print(f'Moving h5 files files for {directory}')
                try: 
                    os.mkdir( os.path.join(subjectdir_date, 'h5Files'))
                    shutil.copy2(h5path, os.path.join(subjectdir_date, 'h5Files'))
                except: print('File already exists. ', directory)
                
                try: shutil.rmtree(subjectdir+'/'+directory+'/'+'h5Files')
                except: pass
            

            # # Check if has psth
            # if haspsth == 1:
            #     # if os.path.isdir(os.listdir(subjectdir+'/'+directory+'/'+'psth')):
            #         # if len(os.listdir(subjectdir+'/'+directory+'/'+'psth'))==0: 
            #     print(f'Copy psth files for {directory}')

            # # check if has nwb file
            # if any(fname.endswith('.nwb') for fname in os.listdir(subjectdir+'/'+directory)):
            #     for fname in os.listdir(subjectdir+'/'+directory): 
            #         if fname.endswith('.nwb'): assert fname == directory+'.nwb', "Check NWB file name."
            # else: print(f'Create NWB file for File {directory}')
                    
        # else: 
        #     ############### Copy Files in Respective Inventory Directory ##################
        #     ###############################################################################
        #     print(f'Copying Files for {directory}')
        #     shutil.copytree(spikepath, os.path.join(subjectdir, directory, 'SpikeTimes'))
        #     if hash5 == 1: 
        #         os.mkdir(os.path.join(subjectdir, directory, 'h5Files'))
                
        #         try: shutil.copy2(h5path, os.path.join(subjectdir, directory, 'h5Files', h5path.split('/')[-1]))
        #         except: print(h5path.split("[")[-1].split("]")[0][0])
        

# df['Path: SpikeTimes'][1]


/braintree/home/aliya277/inventory/norm_FOSS
/braintree/home/aliya277/inventory/norm_FOSS/norm_FOSS.sub_pico
/braintree/home/aliya277/inventory/norm_FOSS/norm_FOSS.sub_pico/norm_FOSS.sub_pico.20220616
Moving h5 files files for norm_FOSS.sub_pico.20220616_111545.proc
/braintree/home/aliya277/inventory/norm_FOSS
/braintree/home/aliya277/inventory/norm_FOSS/norm_FOSS.sub_pico
/braintree/home/aliya277/inventory/norm_FOSS/norm_FOSS.sub_pico/norm_FOSS.sub_pico.20220615
Moving h5 files files for norm_FOSS.sub_pico.20220615_113442.proc
/braintree/home/aliya277/inventory/norm_FOSS
/braintree/home/aliya277/inventory/norm_FOSS/norm_FOSS.sub_pico
/braintree/home/aliya277/inventory/norm_FOSS/norm_FOSS.sub_pico/norm_FOSS.sub_pico.20220907
Moving h5 files files for norm_FOSS.sub_pico.20220907_142157.proc
/braintree/home/aliya277/inventory/norm_FOSS
/braintree/home/aliya277/inventory/norm_FOSS/norm_FOSS.sub_pico
/braintree/home/aliya277/inventory/norm_FOSS/norm_FOSS.sub_pico/norm_FOSS.sub_pico.2022092

In [14]:
h5path

'/braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/h5/230906.pico.rsvp.normalizers.experiment_psth_raw.h5'

In [5]:
file_paths

['/braintree/data2/active/users/sgouldin/jupyter/data_assemblies/02.27-03.03.nc']

In [6]:
from utils.config_helper import create_yaml
import os
dir = '/braintree/home/aliya277/inventory/'
array_meta_path  = '/braintree/data2/active/users/sgouldin/array-metadata'
for files in os.listdir(dir):
    num_files = 0
    path = os.path.join(dir, files)
    try: 
        num_files = len(os.listdir(os.path.join(path,'SpikeTimes')))
    except: pass

    if num_files == 192: 
        array_metadata = os.path.join(array_meta_path, '021023_pico_mapping_noCIT_adapter_version.json')
        adapter_info_avail = True
    elif num_files == 288: 
        array_metadata = os.path.join(array_meta_path,'pico_firstmapping_Lhem_2023.json')
        adapter_info_avail = False
        
        print(path)
    create_yaml(path, array_metadata, adapter_info_avail)

    # except: print(f'Check: {files}')

NameError: name 'array_metadata' is not defined

In [ ]:
import pandas as pd
df_ = pd.read_excel( '/braintree/home/aliya277/dandi_brainscore/pico_inventory.xlsx'  )
df_

In [ ]:
arr, count = np.unique(np.array(df_['ImageSet']), return_counts=True)
sum(count)

In [ ]:
layer0=[]
layer1=[]
layer2=[]
layer3=[]
layer4=[]
layer5=[]
import numpy as np

for spike, h5 , excel, img in zip(df_['Has SpikeTime'], df_['Has h5'], df_['Has Excel'], df_['ImageSet']):
    if img == 'normalizers' or img == 'normalizers-HVM':
        layer0.append(0)
        if excel == 1:
            layer1.append(0)
        else:
            layer1.append(1)
        if spike == 1:
            layer2.append(0)
        else:
            layer2.append(1)
        if h5 == 1:
            layer3.append(0)
        else:
            layer3.append(1)

    else:
        layer0.append(1)
        if excel == 1:
            layer1.append(2)
        else:
            layer1.append(3)
        if spike == 1:
            layer2.append(2)
        else:
            layer2.append(3)
        if h5 == 1:
            layer3.append(2)
        else:
            layer3.append(3)

print('normalizers, rest')
print(np.unique(layer0, return_counts=True))
print('has excel, has not')
print(np.unique(layer1, return_counts=True))
print('has spike, has not')
print(np.unique(layer2, return_counts=True))
print('has h5, has not')
print(np.unique(layer3, return_counts=True))

In [ ]:
import plotly.express as px

# Define the data for the treemap
data = {
    'Category': ['All Recordings', 'Has Excel', 'Has SpikeTime', 'SpikeTime'],
    'Parent': ['', 'All Recordings', 'Has Excel', 'Has Excel'],
    'Value': [400, 300, 100, 100]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Create a treemap
fig = px.treemap(df, names='Category', parents='Parent', values='Value',color_discrete_sequence=px.colors.sequential.Viridis)


fig.update_traces(textinfo="label+text")

# Show the treemap
fig.show()

In [ ]:
import plotly.express as px
# df = df_.drop([259:259])
df_plot = df_
df_plot['Has SpikeTime'].replace({0: 'No SpikeTime', 1:'SpikeTime'}, inplace=True)
df_plot['Has h5'].replace({0: 'No psth', 1:'psth'}, inplace=True)
df_plot['Has Excel'].replace({0: 'No excel', 1:'excel'}, inplace=True)
fig = px.sunburst(df_plot, path=['ImageSet', 'Has Excel', 'Has SpikeTime', 'Has h5'], values='date', width=1000, height=1000)
fig.show()
# df

In [ ]:

import h5py
import os 
import numpy as np 

path = '/braintree/home/aliya277/inventory/exp_gratingsAdap_s5.sub_pico.20230801_144134.proc'
filename = '230801.pico.rsvp.gratingsAdap_s5.experiment_psth_raw.h5'
file = h5py.File(os.path.join(path, 'h5Files', filename),'r+') 
file.close()
# data = file['psth'][:]
# meta = np.array([np.array(file['meta']['start_time_ms']), np.array(file['meta']['stop_time_ms']), np.array(file['meta']['tb_ms'])])
# print(meta)

In [ ]:
import io, os, pynwb
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget
############### Check if All Files are Written and can be Opened ##############
###############################################################################

inventory   = '/braintree/home/aliya277/inventory/'
all_files = os.listdir(inventory)
for folder in all_files[:24]:
        path = os.path.join(inventory, folder)
        try:
                io = NWBHDF5IO(os.path.join(path, f"{folder}.nwb"), "r") 
                nwbfile = io.read()
                io.close()
        except: print(f'This File can not be opened: {folder}')


In [ ]:
############### Validate All Files Using pwnyb, nwbinspectors #################
###############################################################################

import io, os, pynwb
from pynwb import validate
from nwbinspector import inspect_nwbfile

inventory   = '/braintree/home/aliya277/inventory/'
all_nwb_paths = []
all_files = os.listdir(inventory)
for folder in all_files[:24]:
    path = os.path.join(inventory, folder, f'{folder}.nwb')
    all_nwb_paths.append(path)
    


pynwb_validation = validate(paths = all_nwb_paths)

nwbinspector_validation = []
for path in all_nwb_paths:
    results = list(inspect_nwbfile(nwbfile_path=path))
    nwbinspector_validation.append(results)

In [ ]:
from dandi import validate 

dandi = validate.validate(all_nwb_paths[0])
print(dandi)

In [ ]:
nwbinspector_validation

In [ ]:
pynwb_validation

In [ ]:
import io, os, pynwb
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

inventory   = '/braintree/home/aliya277/inventory/'
all_files = os.listdir(inventory)
folder = all_files[0]
path = os.path.join(inventory, folder, f'{folder}.nwb')
io = NWBHDF5IO(path, "r") 
nwbfile = io.read()
# io.close()
nwb2widget(nwbfile)

In [ ]:
nwbfile.subject.sex = 'M'

In [ ]:
import io, os, pynwb
inventory   = '/braintree/home/aliya277/inventory/'
all_nwb_paths = []
all_files = os.listdir(inventory)
for folder in all_files:
    path = os.path.join(inventory, folder, f'{folder}.nwb')
    all_nwb_paths.append(path)
all_files

In [ ]:
from pynwb import validate

validate(paths = all_nwb_paths[0:5])

In [ ]:
from nwbinspector import inspect_nwbfile

for path in all_nwb_paths[0:10]:
    results = list(inspect_nwbfile(nwbfile_path=path))
    print(results)